# Training with HPO using SageMaker Hyperparameter Tuning Job

## 1. Preparation

In [2]:
import sagemaker

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/DEMO-pytorch-mnist'
role = sagemaker.get_execution_role()

In [5]:
from torchvision.datasets import MNIST
from torchvision import transforms

local_dir = 'data'
MNIST.mirrors = ["https://sagemaker-sample-files.s3.amazonaws.com/datasets/image/MNIST/"]
MNIST(
    local_dir,
    download=True,
    transform=transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
    )
)

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )

In [6]:
inputs = sagemaker_session.upload_data(path='data', bucket=bucket, key_prefix=prefix)

In [7]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
    entry_point='mnist.py',
    role=role,
    py_version='py3',
    framework_version='1.8.0',
    instance_count=1,
    instance_type='ml.c5.2xlarge',
    hyperparameters={'epochs': 1, 'backend': 'gloo'},
    source_dir='source_dir'
)

## 2. Training

In [8]:
hyperparameter_ranges = {
    'lr': sagemaker.tuner.ContinuousParameter(0.001, 0.1),
    'batch-size': sagemaker.tuner.CategoricalParameter([32, 64, 128, 256, 512]),
}

In [14]:
tuner = sagemaker.tuner.HyperparameterTuner(
    estimator,
    objective_metric_name='average test loss',
    objective_type='Minimize',
    hyperparameter_ranges=hyperparameter_ranges,
    metric_definitions=[{
        'Name': 'average test loss',
        'Regex': 'Test set: Average loss: ([0-9\\.]+)'
    }],
    max_jobs=9,
    max_parallel_jobs=3,
)

In [ ]:
tuner.fit({'training': inputs})

...........................................................................................................